# Common DataFrame manipulations to all storage options 

In [1]:
import os
from openpyxl import load_workbook
import pandas as pd
from datetime import timedelta

BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath('')))
DATA_DIR = os.path.join(BASE_DIR, 'amerigo_island', 'data')
RAW_DATA_DIR = os.path.join(DATA_DIR, 'raw')
INTERIM_DATA_DIR = os.path.join(DATA_DIR, 'interim')

INPUT_LOAD_AND_VRE_FILENAME = 'load_and_vre.csv'
FINAL_LOAD_AND_VRE_FILENAME = 'final_load_and_vre.csv'

In [2]:
# all input columns as variables for later use
date_colname = 'date'
load_colname = 'load_mw'
solar_gen_colname = 'solar_mw'
wind_gen_colname = 'wind_mw'

load_and_vre_df = pd.read_csv(os.path.join(RAW_DATA_DIR, INPUT_LOAD_AND_VRE_FILENAME))
input_cols = [date_colname, load_colname, wind_gen_colname, solar_gen_colname]
load_and_vre_df = load_and_vre_df.iloc[:, [i for i in range(len(input_cols))]]
load_and_vre_df.columns = input_cols

In [43]:
# all derived columns as variables for later use

### COMMON - columns needed bt legacy gen, batteries, hydrogen
total_vre_gen_colname = 'total_vre_mw'
day_of_month_colname = 'day_of_month'
day_of_yr_colname = 'day_of_yr'
week_of_yr_colname = 'wk_of_yr'
month_colname = 'month'
year_colname = 'year'
weekday_colname = 'weekday'
daily_load_vre_diff_colname = 'daily_load_vre_diff'
prev_7_load_colname = 'prev_7_load'
prev_7_load_vre_diff_colname = 'prev_7_load_vre_diff'
prev_7_vre_gen_colname = 'prev_7_vre_gen'
critical_load_mw_colname = 'critical_load_mw'
critical_load_less_vre_colname = 'critical_load_less_vre_mw'
surplus_vre_colname = 'surplus_vre'
electrolyzer_power_input_mw_colname = 'electrolyzer_power_input_mw'
fuel_cell_power_generation_mw_colname = 'fuel_cell_power_generation_mw'

### BATTERIES 
cuml_load_since_prev_charge_colname = 'cuml_load_since_prev_charge_mw'
cuml_charge_since_prev_charge_colname = 'cuml_charge_since_prev_discharge_mw'
battery_soc_colname = 'battery_soc'

### HYDROGEN
req_hydrogen_kg_colname = 'reqd_hydrogen_kg'
hydrogen_production_kg_colname = 'hydrogen_prod_kg'
hydrogen_production_m3_colname = 'hydrogen_prod_m3'
hydrogen_h2o_demand_l_colname = 'hydrogen_h2o_demand_l'
hydrogen_h2o_demand_m3_colname = 'hydrogen_h2o_demand_m3'
compression_desal_demand_mw_colname = 'compression_desal_demand_mw' 
h2_demand_comp_desal_kg_colname = 'h2_demand_comp_desal_kg'
cuml_h2_storage_kg_colname = 'cuml_h2_storage_kg'
cuml_hydrogen_demand_kg_colname = 'cuml_hydrogen_demand_kg'
cuml_h2_energy_demand_mwh_colname = 'cuml_h2_energy_demand_mwh'

### BAU - LEGACY GENERATION
thermal_gen_total_mw_colname = 'thermal_gen_total_mw'
thermal_gen_rice_mw_colname = 'thermal_gen_rice_mw'
thermal_gen_combustion_mw_colname = 'thermal_gen_combustion_mw'

In [34]:
# other values/assumptions needed for the analysis
CRITICAL_LOAD_PERC = (1/3)
SOLAR_SCALE_FACTOR = 1
WIND_SCALE_FACTOR = 1

SOLAR_INSTALLATION_COST_MM = 50.8
WIND_INSTALLATION_COST_MM = 31.2

In [35]:
def clean_input_columns(df):

    cleaned_df = df.copy()
    cleaned_df = cleaned_df.dropna(how='all')

    cleaned_df[date_colname] = pd.to_datetime(cleaned_df[date_colname])
    # all solar vals need to be > 0
    cleaned_df[solar_gen_colname] = cleaned_df[solar_gen_colname]\
        .apply(lambda x: x if x > 0 else 0)
    
    cleaned_df[month_colname] = cleaned_df[date_colname].map(lambda x: x.month)
    cleaned_df[day_of_month_colname] = cleaned_df[date_colname].map(lambda x: x.day)
    cleaned_df[day_of_yr_colname] = cleaned_df[date_colname].map(lambda x: x.timetuple().tm_yday)
    
    return cleaned_df

In [36]:
cleaned_load_and_vre_df = clean_input_columns(load_and_vre_df)

In [37]:
assert len(cleaned_load_and_vre_df) == 8760, "Expected one row per hour in year (8760), got {} rows".format(len(load_and_gen_df))

for day_of_yr in cleaned_load_and_vre_df[day_of_yr_colname].unique():
    if len(cleaned_load_and_vre_df[cleaned_load_and_vre_df[day_of_yr_colname] == day_of_yr]) != 24:
        assert ValueError("Day {} has {} values".format(day_of_yr, len(cleaned_load_and_vre_df[cleaned_load_and_vre_df[day_of_yr_colname] == day_of_yr])))
        

In [38]:
def add_common_derived_columns(df):
    
    decorated_df = df.copy()
    
    decorated_df[solar_gen_colname] = decorated_df[solar_gen_colname] * SOLAR_SCALE_FACTOR
    decorated_df[wind_gen_colname] = decorated_df[wind_gen_colname] * WIND_SCALE_FACTOR
    
    decorated_df[total_vre_gen_colname] = \
        decorated_df[wind_gen_colname] + decorated_df[solar_gen_colname]
    
    decorated_df[critical_load_mw_colname] = \
        decorated_df[load_colname] * CRITICAL_LOAD_PERC
    
    decorated_df[critical_load_less_vre_colname] = \
        decorated_df[critical_load_mw_colname] - decorated_df[total_vre_gen_colname]
    
    decorated_df[surplus_vre_colname] = \
        -decorated_df[critical_load_less_vre_colname]

    decorated_df[critical_load_less_vre_colname] = decorated_df[critical_load_less_vre_colname]\
        .map(lambda x: x if x > 0 else 0)

    decorated_df[surplus_vre_colname] = decorated_df[surplus_vre_colname]\
        .map(lambda x: x if x > 0 else 0)
    
    return decorated_df

In [39]:
final_load_and_vre_df = add_common_derived_columns(cleaned_load_and_vre_df)
final_load_and_vre_df.to_csv(os.path.join(INTERIM_DATA_DIR, FINAL_LOAD_AND_VRE_FILENAME), index=False)

# Hydrogen-specific analysis

In [47]:
ELEC_EFF_H2_FUEL_CELL_KWH_PER_KG = 19.988333333333
ELEC_EFF_ELECTROLYSIS_KWH_PER_KG = 55.4307635

ENERGY_CONSUMPTION_H2O_DESAL_KWH_PER_M3 = 1.8
DENSITY_H2_300BAR_KG_PER_M3 = 20
DENSITY_H2_30BAR_KG_PER_M3 = 2.38
H2O_ELECTROLYSIS_CONSUMPTION_L_PER_M3 = 1.4

DENSITY_H2_STP_KG_PER_M3 = 0.0813
H2_TAP_WATER_CONSUMPTION_L_PER_M3 = 1.4

ENERGY_CONSUMPTION_H2_KWH_PER_KG = 4
STARTING_H2_IN_STORAGE_KG = 10000

def calc_h2_cols(df):
    
    decorated_df = df.copy()
    
    decorated_df[req_hydrogen_kg_colname] = \
        (decorated_df[critical_load_less_vre_colname] * 1000)/ ELEC_EFF_H2_FUEL_CELL_KWH_PER_KG
    
    decorated_df[hydrogen_production_kg_colname] = \
        (decorated_df[surplus_vre_colname] * 1000) / ELEC_EFF_ELECTROLYSIS_KWH_PER_KG
    
    decorated_df[hydrogen_production_m3_colname] = \
        decorated_df[hydrogen_production_kg_colname] / DENSITY_H2_30BAR_KG_PER_M3
    
    decorated_df[hydrogen_h2o_demand_l_colname] = \
        decorated_df[hydrogen_production_m3_colname] * H2O_ELECTROLYSIS_CONSUMPTION_L_PER_M3
    
    decorated_df[hydrogen_h2o_demand_m3_colname] = \
        (decorated_df[hydrogen_production_kg_colname] / DENSITY_H2_STP_KG_PER_M3) \
        * (H2_TAP_WATER_CONSUMPTION_L_PER_M3 / 1000)
    
    decorated_df[compression_desal_demand_mw_colname] = \
        (decorated_df[hydrogen_production_kg_colname] * ENERGY_CONSUMPTION_H2_KWH_PER_KG / 1000) \
        + (decorated_df[hydrogen_h2o_demand_m3_colname] * ENERGY_CONSUMPTION_H2O_DESAL_KWH_PER_M3 / 1000)
    
    decorated_df[h2_demand_comp_desal_kg_colname] = \
        (decorated_df[compression_desal_demand_mw_colname] * 1000)/ ELEC_EFF_H2_FUEL_CELL_KWH_PER_KG
    
    decorated_df.sort_values(date_colname)
    
# calc cuml h2 demand
    decorated_df[cuml_hydrogen_demand_kg_colname] = None
    decorated_df[cuml_h2_storage_kg_colname] = None
    decorated_df.reset_index()
    for idx, row in decorated_df.iterrows():
        
        if idx == 0:
            h2_demand_val = row[req_hydrogen_kg_colname]
            h2_storage_val = STARTING_H2_IN_STORAGE_KG \
                - row[req_hydrogen_kg_colname] \
                - row[h2_demand_comp_desal_kg_colname] \
                + row[hydrogen_production_kg_colname]
            
            
    
            
        else : #
            
            h2_demand_val = \
                decorated_df.loc[idx - 1, cuml_hydrogen_demand_kg_colname] \
                + row[req_hydrogen_kg_colname]\
                - row[hydrogen_production_kg_colname]
            
            h2_storage_val = decorated_df.loc[idx - 1, cuml_h2_storage_kg_colname] \
                - row[req_hydrogen_kg_colname] \
                - row[h2_demand_comp_desal_kg_colname] \
                + row[hydrogen_production_kg_colname]
        
        decorated_df.at[idx, cuml_hydrogen_demand_kg_colname] = h2_demand_val
        decorated_df.at[idx, cuml_h2_storage_kg_colname] = h2_storage_val
    
        
    decorated_df[electrolyzer_power_input_mw_colname] = \
        decorated_df[hydrogen_production_kg_colname] * ELEC_EFF_ELECTROLYSIS_KWH_PER_KG/ 1000
    decorated_df[cuml_h2_energy_demand_mwh_colname] = \
        (decorated_df[cuml_hydrogen_demand_kg_colname] * ELEC_EFF_H2_FUEL_CELL_KWH_PER_KG) / 1000
    decorated_df[fuel_cell_power_generation_mw_colname] = \
        (decorated_df[req_hydrogen_kg_colname] \
        + decorated_df[h2_demand_comp_desal_kg_colname]) \
        * (ELEC_EFF_H2_FUEL_CELL_KWH_PER_KG / 1000)
    

    return decorated_df

In [48]:
h2_hourly_df = calc_h2_cols(final_load_and_vre_df)

h2_daily_df = h2_hourly_df.groupby(day_of_yr_colname).sum().reset_index()
h2_daily_df = h2_daily_df.drop([month_colname, day_of_month_colname], axis=1)

In [49]:
h2_hourly_df.head(24)

,date,load_mw,wind_mw,solar_mw,month,day_of_month,day_of_yr,total_vre_mw,critical_load_mw,critical_load_less_vre_mw,...,hydrogen_prod_m3,hydrogen_h2o_demand_l,hydrogen_h2o_demand_m3,compression_desal_demand_mw,h2_demand_comp_desal_kg,cuml_hydrogen_demand_kg,cuml_h2_storage_kg,electrolyzer_power_input_mw,cuml_h2_energy_demand_mwh,fuel_cell_power_generation_mw
0,2017-01-01 00:00:00,35.1,1.679660,0.000,1,1,1,1.679660,11.700000,10.020340,...,0.000000,0.000000,0.000000,0.000000,0.000000,501.309,9498.69,0.000000,10.0203,10.020340
1,2017-01-01 01:00:00,35.1,1.679660,0.000,1,1,1,1.679660,11.700000,10.020340,...,0.000000,0.000000,0.000000,0.000000,0.000000,1002.62,8997.38,0.000000,20.0407,10.020340
2,2017-01-01 02:00:00,34.6,1.233270,0.000,1,1,1,1.233270,11.533333,10.300063,...,0.000000,0.000000,0.000000,0.000000,0.000000,1517.92,8482.08,0.000000,30.3407,10.300063
3,2017-01-01 03:00:00,34.2,1.091930,0.000,1,1,1,1.091930,11.400000,10.308070,...,0.000000,0.000000,0.000000,0.000000,0.000000,2033.63,7966.37,0.000000,40.6488,10.308070
4,2017-01-01 04:00:00,33.8,1.465100,0.000,1,1,1,1.465100,11.266667,9.801567,...,0.000000,0.000000,0.000000,0.000000,0.000000,2523.99,7476.01,0.000000,50.4504,9.801567
5,2017-01-01 05:00:00,33.4,1.120290,0.000,1,1,1,1.120290,11.133333,10.013043,...,0.000000,0.000000,0.000000,0.000000,0.000000,3024.94,6975.06,0.000000,60.4634,10.013043
6,2017-01-01 06:00:00,33.3,0.913553,0.000,1,1,1,0.913553,11.100000,10.186447,...,0.000000,0.000000,0.000000,0.000000,0.000000,3534.56,6465.44,0.000000,70.6499,10.186447
7,2017-01-01 07:00:00,33.8,1.841290,0.000,1,1,1,1.841290,11.266667,9.425377,...,0.000000,0.000000,0.000000,0.000000,0.000000,4006.1,5993.9,0.000000,80.0752,9.425377
8,2017-01-01 08:00:00,34.7,1.593750,3.126,1,1,1,4.719750,11.566667,6.846917,...,0.000000,0.000000,0.000000,0.000000,0.000000,4348.64,5651.36,0.000000,86.9222,6.846917
9,2017-01-01 09:00:00,34.7,1.177780,9.952,1,1,1,11.129780,11.566667,0.436887,...,0.000000,0.000000,0.000000,0.000000,0.000000,4370.5,5629.5,0.000000,87.3591,0.436887
